[![View notebook](https://img.shields.io/static/v1?label=render%20on&logo=github&color=87ce3e&message=GitHub)](https://github.com/open-atmos/PySDM/blob/main/examples/PySDM_examples/Gedzelman_and_Arnold_1994/fig_2.ipynb)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/open-atmos/PySDM.git/main?urlpath=lab/tree/examples/PySDM_examples/Gedzelman_and_Arnold_1994/fig_2.ipynb)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/open-atmos/PySDM/blob/main/examples/PySDM_examples/Gedzelman_and_Arnold_1994/fig_2.ipynb)

#### based on Fig. 2 from Gedzelman and Arnold 1994 (JGR)  "_Modeling the isotopic composition of precipitation_"
https://doi.org/10.1029/93JD03518

**notes**:
- "b" defined in eq. (18) lacks multiplication by $\rho_\text{s}$ - units do not match, plot matches the one generated with $\rho_\text{s}$ included - seems a typo in eq. (18)

**extentions (see notebook cells below first figure):**
- multiple isotopes (D, $^{18}$O, $^{17}$O)
- modern parameterisations: Horita & Wesolowski '94 for D & $^{18}$O equil. fractionation, Barkan & Luz '05 for $^{17}$O, Hellmann & Harvey '20 for diffusivities
- environment ratio plot axis
- temperature dependence

In [14]:
import sys
if 'google.colab' in sys.modules:
    !pip --quiet install open-atmos-jupyter-utils
    from open_atmos_jupyter_utils import pip_install_on_colab
    pip_install_on_colab('PySDM')

In [15]:
from open_atmos_jupyter_utils import show_plot
from PySDM import Formulae
from PySDM.physics.constants import si, PER_MILLE, T0, in_unit
from matplotlib import pyplot
import numpy as np

VENTILATION_COEFFICIENT_RATIO = 1  # (this is not taken into account as it depends on droplet size which is not considered here)

TEMPERATURES = np.linspace(T0-10, T0 + 30 * si.K, 9)
DELTAS_2H = (-300 * PER_MILLE, -200 * PER_MILLE, -100 * PER_MILLE)

#### derivations:

- Maxwell-Mason equation (introducing $\rho_\text{s}=\frac{p_\text{vs}}{R_\text{v}T}$, approximated for $S_\text{eq}\approx 1$ and taking into account $\frac{l_\text{v}}{R_\text{v} T} \approx 20 \gg 1$):
$$
a\frac{da}{dt} 
= f \frac{
    S - S_\text{eq}
}{
    \rho_w \left(
        \frac{R_\text{v} T }{\Psi p_\text{vs}(T)} + 
        \frac{l_\text{v}}{K T} \left(\frac{l_\text{v}}{R_\text{v} T}-1\right)
    \right)
}
\approx \frac{f}{\rho_\text{w}} \frac{S - 1}{        \frac{1}{\Psi \rho_\text{s}} + 
        \frac{l^2_\text{v}}{K R_v T^2}
}
= \frac{f \Psi \rho_\text{s}}{\rho_\text{w}} \frac{S - 1}{        1 + 
        \frac{\Psi \rho_\text{s} l^2_\text{v}}{K R_v T^2}
}
$$
where $t$ is time, $a$ is drop radius, $f$ is ventilation factor, $\Psi$ is mass diffusion coefficient, $K$ thermal diffusion coefficient, $l_\text{v}$ is latent heat of condensation, $R_\text{v}$ gas constant for water vapour, $T$ temperature, $\rho_\text{w}$ density of water, $\rho_\text{s}$ saturation vapour density.

In [16]:
CONST = Formulae().constants
print(f"lv/(Rv T) at the triple point: {CONST.l_tri / (CONST.Rv * CONST.T_tri):.3g}")
CONST = None

- expressed in terms of mass $m=\frac{4}{3}\pi a^3 \rho_\text{w}$:

$$\frac{dm}{dt} / [\text{surface}]
= \frac{d\left(\frac{4}{3}\pi a^3 \rho_\text{w}\right)}{dt} \frac{1}{4 \pi a^2}
= \rho_\text{w} \frac{da}{dt}
$$

- introducing $b=\frac{l^2_\text{v} \Psi}{K R_\text{v}T^2}$ and approximating leads to eq. (16) in the paper:

$$
\frac{dm}{dt} 
\approx 4 \pi a \Psi f \rho_\text{s}\frac{S-1}{1 + b \rho_\text{s}}
$$

In [17]:
class Commons:  # pylint: disable=too-few-public-methods
    """ groups values used in both equations """
    def __init__(self, **kwargs):
        const = kwargs['formulae'].constants
        self.vsmow_ratio = getattr(const, f'VSMOW_R_{kwargs["isotope"]}')
        self.iso_ratio_v = kwargs['formulae'].trivia.isotopic_delta_2_ratio(kwargs['delta_v'], self.vsmow_ratio)
        alpha_fun = getattr(
            kwargs['formulae'].isotope_equilibrium_fractionation_factors,
            f'alpha_l_{kwargs["isotope"]}'
        )
        if kwargs['isotope'] == '17O':
            alpha_l_18O = kwargs['formulae'].isotope_equilibrium_fractionation_factors.alpha_l_18O(kwargs['temperature'])
            self.alpha_w = alpha_fun(np.nan, alpha_l_18O)
        else:
            self.alpha_w = alpha_fun(kwargs['temperature'])

        self.diff_coef_ratio = 1 / getattr(
            kwargs['formulae'].isotope_diffusivity_ratios,
            f'ratio_{kwargs["isotope"]}'
        )(kwargs['temperature'])

        missing_b_multiplier = (
            kwargs['formulae'].saturation_vapour_pressure.pvs_Celsius(kwargs['temperature'] - T0)
            / kwargs['temperature']
            / const.Rv
        )
        self.b = (
            missing_b_multiplier
            * kwargs['formulae'].latent_heat.lv(kwargs['temperature'])**2
            * const.D0
            / const.K0
            / const.Rv
            / kwargs['temperature']**2
        )
        self.zero_dR_condition = kwargs['formulae'].isotope_ratio_evolution.zero_dR_condition

class NoFractionationSaturation:  # pylint: disable=too-few-public-methods
    """ embodies eqs. (22) an (23) from the paper """
    def __init__(self, cmn: Commons, *, liquid: bool = False, vapour: bool = False):
        assert liquid != vapour
        self.liquid = liquid
        self.cmn = cmn
        
    def __call__(self, iso_ratio_r):
        return self.cmn.zero_dR_condition(
            iso_ratio_x=iso_ratio_r if self.liquid else self.cmn.iso_ratio_v,
            diff_rat = VENTILATION_COEFFICIENT_RATIO * self.cmn.diff_coef_ratio,
            b=self.cmn.b,
            alpha_w=self.cmn.alpha_w,
            iso_ratio_r=iso_ratio_r,
            iso_ratio_v=self.cmn.iso_ratio_v,
        )

In [18]:
def plot(*, ax, xlim, temperature_label=False, **commons_kwargs):
    commons = Commons(**commons_kwargs)
    x = np.linspace(commons.alpha_w * commons.iso_ratio_v / commons.vsmow_ratio, 1.1, 200)
    y_eq_22 = NoFractionationSaturation(commons, liquid=True)(x * commons.vsmow_ratio)
    y_eq_23 = NoFractionationSaturation(commons, vapour=True)(x * commons.vsmow_ratio)

    temp_01 = (commons_kwargs['temperature'] - min(TEMPERATURES))/(max(TEMPERATURES) - min(TEMPERATURES))
    color = (temp_01, .333, 1-temp_01)

    iso_latex = f"^{{{commons_kwargs['isotope'][:-1]}}}{commons_kwargs['isotope'][-1]}"
    
    ax.plot(x, y_eq_22, label='S$_R$(eq)', color=color)
    ax.plot(x, y_eq_23, label='S$_V$(eq)', color=color, linestyle=':')
    ax.set_title(f"ambient vapour $\\delta{iso_latex}$ = {in_unit(commons_kwargs['delta_v'], PER_MILLE):.3g}‰" + (
        '' if temperature_label else f'\t T={commons_kwargs["temperature"] - T0}°C'
    ))
    ax.set_ylim(0, 1)
    ax.set_yticks(np.linspace(0, 1, 5))
    ax.grid(visible=True, axis='both')
    ax.set_xlabel(f'${iso_latex}$: R$_{{rain}}$ / R$_{{SMOW}}$')
    ax.set_ylabel('S=RH')
    ax.set_xlim(*xlim)

    if temperature_label:
        ax.annotate(
            f' {commons_kwargs["temperature"] - T0:g}°C$\\!\\!\\!\\!\\!\\!$',
            (xlim[-1], y_eq_22[-1] - .015),
            fontsize=8
        )
    return x, {'eq_22': y_eq_22, 'eq_23': y_eq_23}

In [19]:
fig2_x, fig2_y = plot(
    ax=pyplot.gca(),
    formulae=Formulae(
        isotope_equilibrium_fractionation_factors='MerlivatAndNief1967',
        isotope_diffusivity_ratios='Stewart1975',
        isotope_ratio_evolution='GedzelmanAndArnold1994',
    ),
    isotope='2H',
    delta_v= -200 * PER_MILLE,
    temperature= T0 + 10 * si.K,
    xlim=(.8, 1)
)

pyplot.legend()

annotate_kwargs = {'bbox': {'boxstyle': 'round', 'fc': '0.9'}, 'ha': 'center'}
pyplot.annotate(text="vapour ratio\n decreases", xy=(.84, .3), **annotate_kwargs)
pyplot.annotate(text="rain ratio\n decreases", xy=(.96, .7), **annotate_kwargs)
pyplot.annotate(text="both ratios\n increase", xy=(.94, .1), **annotate_kwargs)

show_plot('fig_2.pdf')

In [20]:
formulae = Formulae(
    isotope_equilibrium_fractionation_factors='BarkanAndLuz2005+HoritaAndWesolowski1994',
    isotope_diffusivity_ratios='HellmannAndHarvey2020',
    isotope_meteoric_water_line_excess='BarkanAndLuz2007+Dansgaard1964',
    isotope_ratio_evolution='GedzelmanAndArnold1994',
)
deltas_18O = [formulae.isotope_meteoric_water_line_excess.d18O_of_d2H(d2H) for d2H in DELTAS_2H]
deltas_17O = [formulae.isotope_meteoric_water_line_excess.d17O_of_d18O(d18O) for d18O in deltas_18O]

fig, axs = pyplot.subplots(3, 3, figsize=(10,8), sharex=False, sharey=True, tight_layout=True) 

for ax_idx_iso, iso_label in enumerate(('2H', '18O', '17O')):
    for ax_idx_delta, delta_v in enumerate({
        '2H': DELTAS_2H,
        '18O': deltas_18O,
        '17O': deltas_17O,
    }[iso_label]):
        for i, temperature in enumerate(TEMPERATURES):
            plot(
                ax=axs[ax_idx_iso][ax_idx_delta],
                formulae=formulae,
                isotope=iso_label,
                delta_v=delta_v,
                temperature=temperature,
                xlim={
                    '2H': (.7, 1.1),
                    '18O': (.95, 1.1),
                    '17O': (.95, 1.1),
                }[iso_label],
                temperature_label=i in (0, len(TEMPERATURES)-1),
            )        
        if ax_idx_delta != 0:
            axs[ax_idx_iso][ax_idx_delta].set_ylabel('')
show_plot('plot_grid.pdf')